In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pandas as pd

# 엑셀 파일의 경로
file_path = '/home/jaesung/pulmuone/agent_2/data/12_recat_format_cleaned.xlsx'

# 모든 시트를 로드하고 결합하는 함수
def load_and_concat_sheets(file_path):
    # 엑셀 파일의 모든 시트를 딕셔너리 형태로 로드
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    
    # 모든 시트의 데이터를 담을 리스트
    frames = []
    
    # 각 시트의 데이터를 리스트에 추가
    for sheet_name, data in all_sheets.items():
        data['category'] = sheet_name  # 각 데이터프레임에 시트 이름 열 추가 (선택사항)
        frames.append(data)
    
    # 모든 데이터를 하나의 데이터프레임으로 결합
    result = pd.concat(frames, ignore_index=True)
    
    return result

# 엑셀 파일 로드 및 결합
combined_df = load_and_concat_sheets(file_path)
combined_df.head(2)  # 결합된 데이터프레임의 첫 5행 출력

,REG_NO,item_id,item_nm,food_typ_cd,food_typ_nm,open_yn,reg_id,emp_nm,dept_nm,dept_id,...,mat_nm,origins,prnt_id,contents,reg_dt,mod_dt,reg_dt_1,"fn_level(A.reg_no, B.prnt_id)",ing_typ_nm,category
0,MI-230525-004,7643,반듯한식정통꼬리곰탕,NaN,즉석조리식품,N,183501,김햇뜸,CTO_소스팀,22141,...,소꼬리,호주산,0,11.0,2023-05-25 13:49:06,2023-05-25 14:15:41,2023-05-25 13:47:35,1차,축산물,국물류
1,MI-230525-004,7643,반듯한식정통꼬리곰탕,NaN,즉석조리식품,N,183501,김햇뜸,CTO_소스팀,22141,...,차돌양지,호주산,0,3.6,2023-05-25 13:49:06,2023-05-25 14:15:41,2023-05-25 13:47:35,1차,축산물,국물류


In [3]:
import pandas as pd

# 예제 데이터프레임 (이미지 내용을 기반으로)
data = {
    'item_nm': list(combined_df['item_nm']),
    'category': list(combined_df['category']),
    'food_typ_nm': list(combined_df['food_typ_nm']),
    'mat_nm': list(combined_df['mat_nm']),
}

df = pd.DataFrame(data)

# 'item_nm'으로 그룹화
grouped = df.groupby('item_nm').agg({
    'category': 'first',
    'food_typ_nm': 'first',
    'mat_nm': lambda x: list(x),
}).reset_index()

In [4]:
grouped['mat_nm'] = grouped['mat_nm'].apply(lambda x: list(set(x)))

In [5]:
grouped.head()

,item_nm,category,food_typ_nm,mat_nm
0,'23김치생수제비,면,생면주정처리제품,"[호박산이나트륨, 생생김치건더기, 새우젓분말, 밀가루, 정제소금, 다시마엑기스1호,..."
1,'23년돈코츠라멘,소스류,생면주정처리제품,"[hvphr, 동결건조대파, 정제소금, 로스팅육수, 하얀설탕, 사골농축액, 샘표양조..."
2,'23년매운돈코츠라멘,소스류,생면주정처리제품,"[hvphr, 동결건조대파, 정제소금, 로스팅육수, 하얀설탕, 사골농축액, 샘표양조..."
3,'23맑은소고기어묵전골,소스류,어묵,"[당면, 정백당, 연육명태a, 소맥전분, 밀가루, 비프엑기스gf, 대두단백, 설탕,..."
4,'23물냉면밀키트,면,숙면주정처리,"[혼합메밀가루, 소맥분, 주정, 초절임얼갈이배추, 초절임무우, 겨자소스, 도토리가루..."


In [6]:
grouped.columns = ['item_nm', 'category', 'food_typ_nm', 'original']

In [7]:
import pandas as pd
import itertools

# original 리스트에서 최대 n개의 조합을 뽑고, 나머지를 mat_nm2로 저장하는 함수 정의
def generate_limited_combinations_and_remainder(row, n=3):
    mat_nm_list = eval(row['original'])  # 문자열을 리스트로 변환
    if len(mat_nm_list) < n:  # 리스트의 길이가 n보다 적으면 pass
        return pd.DataFrame()  # 빈 데이터프레임을 반환하여 pass
    
    results = []
    # 1개부터 n개까지의 조합 생성
    for k in range(1, min(n + 1, len(mat_nm_list) + 1)):  # 리스트의 길이와 n 중 최소값까지 조합 생성
        combs = list(itertools.combinations(mat_nm_list, k))  # k개의 항목을 뽑는 모든 조합 생성
        for comb in combs:
            mat_nm1 = comb  # 조합을 mat_nm1으로 선택
            mat_nm2 = [item for item in mat_nm_list if item not in mat_nm1] # mat_nm1에 없는 나머지 항목을 mat_nm2로 저장
            results.append([row['item_nm'], mat_nm1, mat_nm2])  # 결과를 저장
    return pd.DataFrame(results, columns=['item_nm', 'mat_nm1', 'mat_nm2'])

In [8]:
# # 모든 조합에 대해 데이터프레임을 확장
# all_combinations_df = pd.DataFrame(columns=['item_nm', 'mat_nm1', 'mat_nm2'])

# for idx, row in df.iterrows():
#     comb_df = generate_limited_combinations_and_remainder(row, n=3)
#     all_combinations_df = pd.concat([all_combinations_df, comb_df], ignore_index=True)


In [9]:
import pandas as pd

all_combinations_df = pd.read_csv('/home/jaesung/pulmuone/agent_2/vectordb_2/all_cases_df.csv', index_col=0)

In [10]:

len(all_combinations_df)

4374039

In [11]:

all_combinations_df.to_csv('/home/jaesung/pulmuone/agent_2/vectordb_2/all_cases_df.csv')

In [12]:
merged_df = pd.merge(all_combinations_df, grouped, on='item_nm')
merged_df.head(2)

,item_nm,mat_nm1,mat_nm2,category,food_typ_nm,original
0,'23김치생수제비,"('밀가루',)","['변성전분sa2', '변성전분pn', '주정', '밀글루텐', '정제소금', '알...",면,생면주정처리제품,"[호박산이나트륨, 생생김치건더기, 새우젓분말, 밀가루, 정제소금, 다시마엑기스1호,..."
1,'23김치생수제비,"('밀가루',)","['변성전분sa2', '변성전분pn', '주정', '밀글루텐', '정제소금', '알...",면,생면주정처리제품,"[호박산이나트륨, 생생김치건더기, 새우젓분말, 밀가루, 정제소금, 다시마엑기스1호,..."


In [13]:
merged_df = merged_df[['item_nm', 'category', 'food_typ_nm', 'original', 'mat_nm1', 'mat_nm2']]
merged_df.head(2)

,item_nm,category,food_typ_nm,original,mat_nm1,mat_nm2
0,'23김치생수제비,면,생면주정처리제품,"[호박산이나트륨, 생생김치건더기, 새우젓분말, 밀가루, 정제소금, 다시마엑기스1호,...","('밀가루',)","['변성전분sa2', '변성전분pn', '주정', '밀글루텐', '정제소금', '알..."
1,'23김치생수제비,면,생면주정처리제품,"[호박산이나트륨, 생생김치건더기, 새우젓분말, 밀가루, 정제소금, 다시마엑기스1호,...","('밀가루',)","['변성전분sa2', '변성전분pn', '주정', '밀글루텐', '정제소금', '알..."


In [23]:
import os
import pandas as pd
import faiss
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Function to create the output directory if not exists
output_dir = 'vectordb_2/categorywise/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# KosimCSE 모델 및 토크나이저 로드 (모델명 GPU 이동)
model = AutoModel.from_pretrained('BM-K/KosimCSE-bert-multitask').cuda()
tokenizer = AutoTokenizer.from_pretrained('BM-K/kosimCSE-bert-multitask')

def get_embeddings_in_batches(texts, batch_size=8192):
    dataset = TextDataset(texts)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    embeddings_list = []
    model.eval()

    with torch.no_grad():
        for batch_texts in tqdm(data_loader, desc="Processing batches", leave=True):
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to('cuda')

            # Mixed Precision 사용해 배치 연산 수행
            with torch.cuda.amp.autocast():  # FP16에서 실행 후 추출
                outputs = model(**inputs).last_hidden_state[:, 0, :]  # 문장 임베딩 추출
                embeddings = outputs.cpu().numpy()

            embeddings_list.append(embeddings)

    return np.vstack(embeddings_list)

/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
merged_df[merged_df['food_typ_nm'].isnull()].head(2)

,item_nm,category,food_typ_nm,original,mat_nm1,mat_nm2
1088467,갈릭굴소스볶음소스&건더기,소스류,None,"[흑후추분말, 굴농축액, 정제소금, 생강엑기스, 건마늘분말, 하얀설탕, 조미맛분, ...","('하얀설탕',)","['샘표양조간장', '고감미물엿82', '굴농축액', '마늘', '미림', '야채풍..."
1088468,갈릭굴소스볶음소스&건더기,소스류,None,"[흑후추분말, 굴농축액, 정제소금, 생강엑기스, 건마늘분말, 하얀설탕, 조미맛분, ...","('샘표양조간장',)","['하얀설탕', '고감미물엿82', '굴농축액', '마늘', '미림', '야채풍미유..."


In [24]:
# # For each unique food type
# for food_type in merged_df['food_typ_nm'].unique():
#     print(food_type)

#     if food_type is None:
#         continue
    
#     # Filter the dataframe for the current food type
#     filtered_df = merged_df[merged_df['food_typ_nm'] == food_type]

#     # Combine the food type and mat_nm1 to create the 'combined_str'
#     texts = filtered_df.apply(lambda x: ' '.join([x['food_typ_nm']] + list(x['mat_nm1'])), axis=1).tolist()
#     metadata = filtered_df['original'].tolist()

#     # Generate embeddings
#     embeddings = get_embeddings_in_batches(texts)

#     # FAISS Index 생성
#     d = embeddings.shape[1]  # 차원수 (768)
#     index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성
#     index.add(embeddings)  # 임베딩 추가

#     # Save FAISS index and metadata
#     food_type_cleaned = food_type.replace('/', '_')  # Clean food type for file names
#     faiss.write_index(index, os.path.join(output_dir, f'{food_type_cleaned}_faiss_index.bin'))

#     # Save metadata as CSV
#     pd.DataFrame(metadata).to_csv(os.path.join(output_dir, f'{food_type_cleaned}_metadata.csv'), index=False)

#     print(f"Saved FAISS index and metadata for food type: {food_type}")

생면주정처리제품


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

Saved FAISS index and metadata for food type: 생면주정처리제품
어묵


Processing batches:   0%|          | 0/19 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZE

Saved FAISS index and metadata for food type: 어묵
숙면주정처리


...

Saved FAISS index and metadata for food type: 유지류
과채가공품류


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

Saved FAISS index and metadata for food type: 과채가공품류
일반환자용균형영양조제식품


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

Saved FAISS index and metadata for food type: 일반환자용균형영양조제식품


In [28]:
# # For each unique food type
# for food_type in merged_df['category'].unique():
#     # Filter the dataframe for the current food type
#     filtered_df = merged_df[merged_df['category'] == food_type]
    
#     # Combine the food type and mat_nm1 to create the 'combined_str'
#     texts = filtered_df.apply(lambda x: ' '.join([x['category']] + list(x['mat_nm1'])), axis=1).tolist()
#     metadata = filtered_df['original'].tolist()

#     # Generate embeddings
#     embeddings = get_embeddings_in_batches(texts)

#     # FAISS Index 생성
#     d = embeddings.shape[1]  # 차원수 (768)
#     index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성
#     index.add(embeddings)  # 임베딩 추가

#     # Save FAISS index and metadata
#     food_type_cleaned = food_type.replace('/', '_')  # Clean food type for file names
#     faiss.write_index(index, os.path.join(output_dir, f'{food_type_cleaned}_faiss_index.bin'))

#     # Save metadata as CSV
#     pd.DataFrame(metadata).to_csv(os.path.join(output_dir, f'{food_type_cleaned}_metadata.csv'), index=False)

#     print(f"Saved FAISS index and metadata for food type: {food_type}")

Processing batches:   0%|          | 0/4 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

Saved FAISS index and metadata for food type: 면


Processing batches:   0%|          | 0/136 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZ

Saved FAISS index and metadata for food type: 소스류


Processing batches:   0%|          | 0/22 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZE

...

Saved FAISS index and metadata for food type: 만두


Processing batches:   0%|          | 0/7 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

Saved FAISS index and metadata for food type: 국물류


Processing batches:   0%|          | 0/5 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

Saved FAISS index and metadata for food type: 제과제빵


In [ ]:
# check out vectordb

## v1.make vectorDB

In [33]:
from dotenv import load_dotenv

load_dotenv()

True

In [79]:
import pandas as pd

# csv 파일을 데이터프레임으로 로드
df = pd.read_csv('/home/jaesung/pulmuone/agent_2/data/combined_data.csv', index_col=False)
df = df[['item_nm', 'food_typ_nm', 'mat_nm']]
df.columns = ['item_nm', 'food_typ_nm', 'original']

In [80]:
df.head()

,item_nm,food_typ_nm,original
0,'23김치생수제비,생면주정처리제품,"['밀가루', '밀가루', '변성전분sa2', '변성전분pn', '주정', '밀글루..."
1,'23맑은소고기어묵전골,어묵,"['대두유', '유산균배양액혼합분말', '정제염', '마늘', '양파', '대두단백..."
2,'23물냉면밀키트,숙면주정처리,"['소맥분', '고구마전분', '혼합메밀가루', '주정', '흑미가루', '정제소금..."
3,'23진한가쓰오어묵전골,어묵,"['대두유', '유산균배양액혼합분말', '정제염', '마늘', '양파', '연육명태..."
4,'23풀스키즈_사각쑥절편,떡류,"['멥쌀', '냉동쑥', '한주꽃소금', '쑥가루', '대두유', '정제수']"


In [35]:
# import pandas as pd
# import itertools

# # original 리스트에서 최대 n개의 조합을 뽑고, 나머지를 mat_nm2로 저장하는 함수 정의
# def generate_limited_combinations_and_remainder(row, n=3):
#     mat_nm_list = eval(row['original'])  # 문자열을 리스트로 변환
#     if len(mat_nm_list) < n:  # 리스트의 길이가 n보다 적으면 pass
#         return pd.DataFrame()  # 빈 데이터프레임을 반환하여 pass
    
#     results = []
#     # 1개부터 n개까지의 조합 생성
#     for k in range(1, min(n + 1, len(mat_nm_list) + 1)):  # 리스트의 길이와 n 중 최소값까지 조합 생성
#         combs = list(itertools.combinations(mat_nm_list, k))  # k개의 항목을 뽑는 모든 조합 생성
#         for comb in combs:
#             mat_nm1 = comb  # 조합을 mat_nm1으로 선택
#             mat_nm2 = [item for item in mat_nm_list if item not in mat_nm1]  # mat_nm1에 없는 나머지 항목을 mat_nm2로 저장
#             results.append([row['item_nm'], mat_nm1, mat_nm2])  # 결과를 저장
#     return pd.DataFrame(results, columns=['item_nm', 'mat_nm1', 'mat_nm2'])

# # 모든 조합에 대해 데이터프레임을 확장
# all_combinations_df = pd.DataFrame(columns=['item_nm', 'mat_nm1', 'mat_nm2'])
# for idx, row in df.iterrows():
#     comb_df = generate_limited_combinations_and_remainder(row, n=3)
#     all_combinations_df = pd.concat([all_combinations_df, comb_df], ignore_index=True)

# all_combinations_df.to_csv('vectordb/')


In [36]:
import pandas as pd

all_combinations_df = pd.read_csv('/home/jaesung/pulmuone/agent_2/vectordb/all_combinations_df', index_col=0)

In [37]:
merged_df = pd.merge(all_combinations_df, df, on='item_nm')
merged_df.head(2)

,item_nm,mat_nm1,mat_nm2,food_typ_nm,original
0,'23김치생수제비,"('밀가루',)","['변성전분sa2', '변성전분pn', '주정', '밀글루텐', '정제소금', '알...",생면주정처리제품,"['밀가루', '밀가루', '변성전분sa2', '변성전분pn', '주정', '밀글루..."
1,'23김치생수제비,"('밀가루',)","['변성전분sa2', '변성전분pn', '주정', '밀글루텐', '정제소금', '알...",생면주정처리제품,"['밀가루', '밀가루', '변성전분sa2', '변성전분pn', '주정', '밀글루..."


In [38]:
# merged_df['food_typ_nm'].unique() 

"""
'생면주정처리제품', '어묵', '숙면주정처리', '떡류', '과채주스', '농산물', '혼합음료', '액상차',
       '홍삼음료', '두유', '즉석조리식품', '빙과', '당뇨환자용식단형식품', '소스', '조미김',
       '기타수산물가공품', '가공두부', '소스살균', '빵류', '소스류', '절임식품', '기타가공품', '절임류',
       '면류', '파스타류숙면살균', '즉석섭취식품', '과자', '가공치즈', '과채음료', '건강기능식품', '만두류',
       '육수', '어육소시지', '생면', '두류가공품', '만두', '숙면살균', '혼합장', '양념육',
       '소스가열섭취냉동식품', '수산물가공품가열섭취냉동식품', '소시지', '두부', '유음료', '분쇄가공육제품',
       '땅콩또는견과류가공품', '수산물가공품', '복합조미식품', '조림식품', '묵류', '신선편의식품', '잼류',
       '식육함유가공품', '분쇄가공육', '알가공품', '김치류', '소스류/살균제품', '국수', '건면',
       '땅콩및견과류가공품', '우양냉동식품', '즉석조리식품가열섭취냉동식품', '숙면가열섭취냉동식품', '당면',
       '어묵유탕처리비살균', '알가열제품', '분쇄가공육제품비살균', '가공유저지방', '빵류비가열섭취냉동식품',
       '식육추출가공품', '햄', '닭식용란', '숙면', '기타가공품가열섭취냉동식품', '당류가공품',
       '체중조절용조제식품', '캔디류', '과채주스비가열함유', '빵류비가열섭취냉동제품', '수산물조림', '견과류가공품',
       '향미유', '자연상태식품', '소스류가열섭취냉동식품', '발효유', '묵류살균제품', '수산물', '조림류',
       '연성치즈', '자연치즈', '빵류핫도그', '염지란', '즉석조리제품', '파스타류살균', '과채가공품', '샤베트',
       '빙과류', '인삼홍삼음료', '생치즈', '드레싱', '즉석조리', '냉면', '유산균음료', '소스류살균',
       '혼합식육추출가공품', '햄류', '국수주정처리', '반경성치즈', '과채가공품류', '두류가공품멸균',
       '빵류가열섭취냉동식품', '과채음료비가열함유', '파스타류', '어묵유탕처리', '농후발효유', '잼',
       '절임류오이무혼합살균', '과자유탕처리', '강화저지방우유', '간편조리세트', '두부류또는묵류', '과채주스비가열',
       '치즈', '즉석조리식품살균', '커피', '수산물조림류', '기타농산가공품', '식초절임살균', '알가열성형제품',
       '카레', '곡류가공품', '건강기능식품신', '소시지살균', '탄산음료', '기타수산물가공품가열섭취냉동식품',
       '체중조절식', '고형차', '국수가열섭취냉동식품', '마요네즈', '가공유', '강화우유', '유산균음료살균',
       '토마토케첩', '가공유류', '갈비가공품', '단순식육추출가공품', '튀김식품', '햄살균', '유지류',
       '일반환자용균형영양조제식품'
"""

"\n'생면주정처리제품', '어묵', '숙면주정처리', '떡류', '과채주스', '농산물', '혼합음료', '액상차',\n       '홍삼음료', '두유', '즉석조리식품', '빙과', '당뇨환자용식단형식품', '소스', '조미김',\n       '기타수산물가공품', '가공두부', '소스살균', '빵류', '소스류', '절임식품', '기타가공품', '절임류',\n       '면류', '파스타류숙면살균', '즉석섭취식품', '과자', '가공치즈', '과채음료', '건강기능식품', '만두류',\n       '육수', '어육소시지', '생면', '두류가공품', '만두', '숙면살균', '혼합장', '양념육',\n       '소스가열섭취냉동식품', '수산물가공품가열섭취냉동식품', '소시지', '두부', '유음료', '분쇄가공육제품',\n       '땅콩또는견과류가공품', '수산물가공품', '복합조미식품', '조림식품', '묵류', '신선편의식품', '잼류',\n       '식육함유가공품', '분쇄가공육', '알가공품', '김치류', '소스류/살균제품', '국수', '건면',\n       '땅콩및견과류가공품', '우양냉동식품', '즉석조리식품가열섭취냉동식품', '숙면가열섭취냉동식품', '당면',\n       '어묵유탕처리비살균', '알가열제품', '분쇄가공육제품비살균', '가공유저지방', '빵류비가열섭취냉동식품',\n       '식육추출가공품', '햄', '닭식용란', '숙면', '기타가공품가열섭취냉동식품', '당류가공품',\n       '체중조절용조제식품', '캔디류', '과채주스비가열함유', '빵류비가열섭취냉동제품', '수산물조림', '견과류가공품',\n       '향미유', '자연상태식품', '소스류가열섭취냉동식품', '발효유', '묵류살균제품', '수산물', '조림류',\n       '연성치즈', '자연치즈', '빵류핫도그', '염지란', '즉석조리제품', '파스타류살균', '과채가공품', '샤베트',\n       '빙과류', '인삼홍삼

In [39]:
import os
import pandas as pd
import faiss
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Function to create the output directory if not exists
output_dir = 'vectordb/categorywise/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# KosimCSE 모델 및 토크나이저 로드 (모델명 GPU 이동)
model = AutoModel.from_pretrained('BM-K/KosimCSE-bert-multitask').cuda()
tokenizer = AutoTokenizer.from_pretrained('BM-K/kosimCSE-bert-multitask')

def get_embeddings_in_batches(texts, batch_size=8192):
    dataset = TextDataset(texts)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    embeddings_list = []
    model.eval()

    with torch.no_grad():
        for batch_texts in tqdm(data_loader, desc="Processing batches", leave=True):
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to('cuda')

            # Mixed Precision 사용해 배치 연산 수행
            with torch.cuda.amp.autocast():  # FP16에서 실행 후 추출
                outputs = model(**inputs).last_hidden_state[:, 0, :]  # 문장 임베딩 추출
                embeddings = outputs.cpu().numpy()

            embeddings_list.append(embeddings)

    return np.vstack(embeddings_list)

# # For each unique food type
# for food_type in merged_df['food_typ_nm'].unique():
#     # Filter the dataframe for the current food type
#     filtered_df = merged_df[merged_df['food_typ_nm'] == food_type]
    
#     # Combine the food type and mat_nm1 to create the 'combined_str'
#     texts = filtered_df.apply(lambda x: ' '.join([x['food_typ_nm']] + list(x['mat_nm1'])), axis=1).tolist()
#     metadata = filtered_df['original'].tolist()

#     # Generate embeddings
#     embeddings = get_embeddings_in_batches(texts)

#     # FAISS Index 생성
#     d = embeddings.shape[1]  # 차원수 (768)
#     index = faiss.IndexFlatL2(d)  # L2 거리 기반 인덱스 생성
#     index.add(embeddings)  # 임베딩 추가

#     # Save FAISS index and metadata
#     food_type_cleaned = food_type.replace('/', '_')  # Clean food type for file names
#     faiss.write_index(index, os.path.join(output_dir, f'{food_type_cleaned}_faiss_index.bin'))

#     # Save metadata as CSV
#     pd.DataFrame(metadata).to_csv(os.path.join(output_dir, f'{food_type_cleaned}_metadata.csv'), index=False)

#     print(f"Saved FAISS index and metadata for food type: {food_type}")


/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## call vectorDB

In [41]:
import os
import faiss
import pandas as pd

# 카테고리 파일 경로 지정
category_dir = 'vectordb/categorywise'

# 쿼리 첫 번째 요소로 카테고리 파일 선택
# query = "떡류, 맵쌀, 가당백앙금, 냉동쑥"
# query = "유음료, 원유, 정제수, 가는정백당"
query = "빵류, 딸기, 생크림"
# query = "빵류, 전란액, 하얀설탕, 박력분"
first_element = query.split(',')[0].strip()  # 쿼리의 첫 번째 요소를 선택하고 공백 제거

# 카테고리 파일이 있는지 확인 (동적으로 파일 탐색)
index_file = os.path.join(category_dir, f"{first_element}_faiss_index.bin")
metadata_file = os.path.join(category_dir, f"{first_element}_metadata.csv")

pd.read_csv(metadata_file).head()

if os.path.exists(index_file) and os.path.exists(metadata_file):
    # FAISS 인덱스와 메타데이터 불러오기
    index = faiss.read_index(index_file)
    meta_df = pd.read_csv(metadata_file)
    meta_df.columns = ['original']
    metadatas = meta_df['original'].tolist()

    # 쿼리 임베딩 생성
    query_embedding = get_embeddings_in_batches([query])

    # FAISS 인덱스에서 유사도 검색
    D, I = index.search(query_embedding, k=1)

    # 검색된 메타데이터 출력
    print(f"가장 유사한 original 값: {metadatas[I[0][0]]}")
else:
    print(f"{first_element}에 해당하는 파일이 없습니다.")


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

가장 유사한 original 값: ['밀가루', '쥬르드뱅페이스트리', '백설탕', '신선전란액살균', '커스타드크림믹스', '호밀가루', '정제소금', '크림치즈', '화인휘프500', '블루베리리플잼', '그랜드500마가린', 'ts탈지분', 'ssdgold', 'skimo500', '매지믹스다크블루']


## call LLM

In [42]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser   #출력을 파싱하여 문자열 형태로 만드는 클래스
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  #langchain_core.prompts로부터 프롬프트를 관리하는 클래스를 가져옮
from langchain.chat_models import ChatOpenAI

# Ollama 로드 : LangChain의 챗모델 
# llm = ChatOllama(model="EEVE-Korean-10.8B:latest")
llm = ChatOpenAI(
        api_key="ollama",
        # model="EEVE-Korean-10.8B:latest",
        # model="bnksys/eeve:10.8b-korean-instruct-q8-v1",
        model="bnksys/eeve:10.8b-korean-instruct-f16-v1",
        # model="bnksys/eeve:korean-instruct",
        base_url="http://localhost:11434/v1",
        temperature=0,
        streaming=True    
    ) 

# 프롬프트를 생성 : 사용자가 주어진 주제에 대해 설명하도록 지시하는 프롬프트를 만듭니다.
prompt_prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 체인 구성 : 사용자의 입력을 받아 챗 모델을 거쳐 파싱된 출력
chain = prompt_prompt | llm | StrOutputParser()

In [43]:
# 채팅 프롬프트 생성 : ChatPromptTemplate의 from_messages 메소드를 사용
chat_prompt = ChatPromptTemplate.from_messages(
    [
        # 이 배열의 첫 번째 요소는 시스템 메시지입니다. 이것은 AI에게 명령을 내립니다.
        # 이 경우에는, 내 이름은 '테디'이고 도움을 주는 AI Assistant임을 명시하고 있습니다. 이 AI는 반드시 한국어로 대답해야 합니다.
        (
            "system",
            "You are a helpful AI Assistant. You must answer in Korean.",
        ),
        # 이 프롬프트는 사용자의 메시지를 저장하는 Placeholder를 포함하고 있습니다.
        # Placeholder는 템플릿에서 변하는 부분을 나타내는데 사용됩니다.
        # 'variable_name' 속성의 값으로 'messages'를 설정하여, 채팅 대화에서 사용자의 메시지를 포함하도록 합니다.
        MessagesPlaceholder(variable_name="messages")
    ]
)

# 'chat_prompt', 'llm', 그리고 'StrOutputParser'를 '|' 연산자를 이용하여 체인으로 연결합니다.
chat_chain = chat_prompt | llm | StrOutputParser()

In [44]:
from langchain_community.llms import Ollama


instruction = f"""
너는 식품 개발자야. 다음과 같은 순서로 식품을 개발해줘.
1. 원본 식재료의 특성을 유지하며 식재료 리스트에 대체 식재료를 추가해줘. 이때, {query.split(', ')[1:]}은 반드시 포함되어야 해.
2. 배합비의 합이 100%가 되게 재계산 해줘. 
3. 출력 형식은 example들과 반드시 같도록 수정해줘.
"""

# Q와 A를 모두 제공하는 예시 (형식 강화)
example_1 = """Q: 원본 식재료 ['토마토', '아보카도', '적양파', '바질']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
A: 레드 벨 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%"""

example_2 = """Q: 원본 식재료 ['올리브 오일', '발사믹 식초', '레몬즙']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
A: 올리브 오일: 50%, 레드 와인 식초: 30%, 시트러스 주스: 20%"""

example_3 = """Q: 원본 식재료 ['감자', '당근', '양파', '마늘', '파슬리', '버터']의 카테고리는 스튜야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
A: 감자: 35%, 당근: 20%, 샬롯: 15%, 생강: 10%, 고수: 10%, 올리브 오일: 10%"""

example_4 = """Q: 원본 식재료 ['돼지고기', '간장', '설탕', '생강', '파', '마늘']의 카테고리는 조림류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
A: 닭고기: 40%, 간장: 20%, 꿀: 15%, 생강: 10%, 샬롯: 10%, 미림: 5%"""

example_5 = """Q: 원본 식재료 ['쌀', '김', '참기름', '계란', '간장', '소금']의 카테고리는 밥류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
A: 현미: 30%, 김: 15%, 들기름: 20%, 메추리알: 10%, 간장: 15%, 소금: 10%"""


# 사용자의 입력
user_input = f"""Q: 원본 식재료 {metadatas[I[0][0]]}의 카테고리는 {first_element}야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100% 로 맞춰줘."""

response = llm.invoke(f"{instruction}\n{example_1}\n{example_2}\n{example_3}\n{example_4}\n{example_5}\n{user_input}")

print(response)

content="\r\nQ: 원본 식재료 ['밀가루', '쥬르드뱅페이스트리', '백설탕', '신선전란액살균', '커스타드크림믹스', '호밀가루', '정제소금', '크림치즈', '화인휘프500', '블루베리리플잼', '그랜드500마가린', 'ts탈지분', 'ssdgold', 'skimo500', '매지믹스다크블루']의 카테고리는 빵류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100% 로 맞춰줘.\r\n\r\nA: 밀가루: 40%, 호밀가루: 25%, 통밀가루: 15%, 설탕: 10%, 소금: 5%, 효모: 5%" additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-06ba4841-625f-4d34-bbd7-8cd8943084c6-0'


In [29]:
response.content.split(':A')[-1].strip()

"Q: 원본 식재료 ['액상과당', '유자과즙', '배농축액', '대추농축액', '구연산', '구연산삼나트륨', '비타민b1염산염', '정제수', '효소처리스테비아', '유자향', '화이바솔2l', '액상과당', '유자과즙', '배농축액', '대추농축액', '구연산', '구연산삼나트륨', '비타민b1염산염', '정제수', '효소처리스테비아', '유자향', '화이바솔2l']의 카테고리는 과채음료야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100% 로 맞춰줘.\r\n\r\nA: 사과주스 농축액: 45%, 라즈베리퓨레: 20%, 레몬즙: 10%, 설탕: 10%, 구연산: 5%, 비타민b1염산염: 5%, 정제수: 15%"

In [69]:
from langchain_community.llms import Ollama

import re

tmp_inging = []
A_in_res = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', response.content.split(':A')[-1].strip())
for inging in query.split(', ')[1:]:
    if inging not in A_in_res:
        tmp_inging.append(inging)

instruction = f"""
너는 식품개발자야.
예제와 같은 형태로 출력해줘.
"""

example_1 = """Q: 원본 식재료 ['토마토', '아보카도', '적양파', '바질']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
A: 레드 벨 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%"""

user_input = f"""Q: {', '.join(A_in_res)}에 {', '.join(tmp_inging)}을 추가해서 배합비를 다시 생성하고, 배합비를 100%로 맞춰줘."""

# 사용자의 입력
# user_input = f"""Q: 사과주스 농축액: 45%, 유자과즙: 30%, 배농축액: 10%, 대추농축액: 5%, 구연산: 5%, 비타민b1염산염: 2%, 정제수: 3% 에 '백설탕'과 '납작복숭아과즙'을 추가해서 배합비를 다시 생성해줘."""

response = llm.invoke(f"{instruction}\n{example_1}\n{user_input}")


print(list(set(A_in_res)))
print('')
print(user_input)
print('')
print(tmp_inging)
print('')
print(response)

['적양파: 20%', '아보카도: 30%', '페퍼: 40%', '파슬리: 10%']

Q: 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%에 정제수, 백설탕, 납작복숭아과즙을 추가해서 배합비를 다시 생성하고, 배합비를 100%로 맞춰줘.

['정제수', '백설탕', '납작복숭아과즙']

content="\r\nQ: 원본 식재료 ['토마토', '아보카도', '적양파', '바질']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\n\r\nA: 레드 벨 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%\r\n\r\nQ: 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%에 정제수, 백설탕, 납작복숭아과즙을 추가해서 배합비를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\n\r\nA: 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%, 정제수: 5%, 백설탕: 5%, 납작복숭아과즙: 5%" additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-2b0b36fe-596c-49a7-91e3-a81bd10f0519-0'


In [17]:
result = """예시1
입력: 떡류, 멥쌀, 가당백앙금, 냉동쑥
출력: 멥쌀: 45%, 가당백앙금: 20%, 냉동쑥: 15%, 파프리카 씨앗: 10%, 대두유: 5%, 정제수: 5%

예시2
입력: 유음료, 원유, 정제수, 가는정백당
출력: 우유: 40%, 정제수: 20%, 설탕: 10%, 생크림: 10%, 옥수수 전분: 5%, 참깨 페이스트: 5%, 현미 가루: 5%, 렌틸콩 가루: 5%, 원유: 5%, 가는정백당: 5%
특이사항: 배합비 안 맞음. 후처리 해야함.

예시3
입력: 과채음료, 정제수, 백설탕, 납작복숭아과즙
출력: 사과주스 농축액: 45%, 유자과즙: 30%, 배농축액: 10%, 대추농축액: 5%, 구연산: 5%, 비타민b1염산염: 2%, 정제수: 3%, 백설탕: 7.5%, 납작복숭아과즙: 7.5%
특이사항: 배합비 안 맞음. 후처리 해야함.

예시4
입력: 빵류, 전락액, 하얀설탕, 박력분
출력: 설탕: 25%, 식물성 기름: 20%, 전란액: 10%, 통밀가루: 20%, 저당: 10%, 베이킹 파우더: 10%, 홍차 가루: 5%, 카라멜 향: 5%, 소금: 5%, 아몬드 조각: 5%, 밀가루: 5%, 
다목적용 박력분: 5%, 전란액: 5%, 설탕: 5%, 소금: 5%, 저당: 5%, 홍차 가루: 5%, 식물성 기름: 5%, 아몬드 조각: 5%, 베이킹 파우더: 5%
특이사항1: 배합비 안 맞음. 후처리 해야함.
특이사항2: 중복되는 요소들이 있음. """

print(result)

예시1
입력: 떡류, 멥쌀, 가당백앙금, 냉동쑥
출력: 멥쌀: 45%, 가당백앙금: 20%, 냉동쑥: 15%, 파프리카 씨앗: 10%, 대두유: 5%, 정제수: 5%

예시2
입력: 유음료, 원유, 정제수, 가는정백당
출력: 우유: 40%, 정제수: 20%, 설탕: 10%, 생크림: 10%, 옥수수 전분: 5%, 참깨 페이스트: 5%, 현미 가루: 5%, 렌틸콩 가루: 5%, 원유: 5%, 가는정백당: 5%
특이사항: 배합비 안 맞음. 후처리 해야함.

예시3
입력: 과채음료, 정제수, 백설탕, 납작복숭아과즙
출력: 사과주스 농축액: 45%, 유자과즙: 30%, 배농축액: 10%, 대추농축액: 5%, 구연산: 5%, 비타민b1염산염: 2%, 정제수: 3%, 백설탕: 7.5%, 납작복숭아과즙: 7.5%
특이사항: 배합비 안 맞음. 후처리 해야함.

예시4
입력: 빵류, 전락액, 하얀설탕, 박력분
출력: 설탕: 25%, 식물성 기름: 20%, 전란액: 10%, 통밀가루: 20%, 저당: 10%, 베이킹 파우더: 10%, 홍차 가루: 5%, 카라멜 향: 5%, 소금: 5%, 아몬드 조각: 5%, 밀가루: 5%, 
다목적용 박력분: 5%, 전란액: 5%, 설탕: 5%, 소금: 5%, 저당: 5%, 홍차 가루: 5%, 식물성 기름: 5%, 아몬드 조각: 5%, 베이킹 파우더: 5%
특이사항1: 배합비 안 맞음. 후처리 해야함.
특이사항2: 중복되는 요소들이 있음. 


In [37]:
print(first_element)
print(metadatas[I[0][0]])

떡류
['멥쌀', '가당백앙금', '냉동쑥', '한주꽃소금', '쑥가루', '대두유', '정제수']


## v2. call LLM

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pandas as pd

# csv 파일을 데이터프레임으로 로드
df = pd.read_csv('/home/jaesung/pulmuone/agent_2/data/combined_data.csv', index_col=False)
df = df[['item_nm', 'food_typ_nm', 'mat_nm']]
df.columns = ['item_nm', 'food_typ_nm', 'original']

In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser   #출력을 파싱하여 문자열 형태로 만드는 클래스
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  #langchain_core.prompts로부터 프롬프트를 관리하는 클래스를 가져옮
from langchain.chat_models import ChatOpenAI

# Ollama 로드 : LangChain의 챗모델 
# llm = ChatOllama(model="EEVE-Korean-10.8B:latest")
llm = ChatOpenAI(
        api_key="ollama",
        # model="EEVE-Korean-10.8B:latest",
        # model="bnksys/eeve:10.8b-korean-instruct-q8-v1",
        model="bnksys/eeve:10.8b-korean-instruct-f16-v1",
        # model="bnksys/eeve:korean-instruct",
        base_url="http://localhost:11434/v1",
        temperature=0,
        streaming=True    
    ) 

# 프롬프트를 생성 : 사용자가 주어진 주제에 대해 설명하도록 지시하는 프롬프트를 만듭니다.
prompt_prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 체인 구성 : 사용자의 입력을 받아 챗 모델을 거쳐 파싱된 출력
chain = prompt_prompt | llm | StrOutputParser()

/tmp/ipykernel_2862639/2418278748.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
# 채팅 프롬프트 생성 : ChatPromptTemplate의 from_messages 메소드를 사용
chat_prompt = ChatPromptTemplate.from_messages(
    [
        # 이 배열의 첫 번째 요소는 시스템 메시지입니다. 이것은 AI에게 명령을 내립니다.
        # 이 경우에는, 내 이름은 '테디'이고 도움을 주는 AI Assistant임을 명시하고 있습니다. 이 AI는 반드시 한국어로 대답해야 합니다.
        (
            "system",
            "You are a helpful AI Assistant. You must answer in Korean.",
        ),
        # 이 프롬프트는 사용자의 메시지를 저장하는 Placeholder를 포함하고 있습니다.
        # Placeholder는 템플릿에서 변하는 부분을 나타내는데 사용됩니다.
        # 'variable_name' 속성의 값으로 'messages'를 설정하여, 채팅 대화에서 사용자의 메시지를 포함하도록 합니다.
        MessagesPlaceholder(variable_name="messages")
    ]
)

# 'chat_prompt', 'llm', 그리고 'StrOutputParser'를 '|' 연산자를 이용하여 체인으로 연결합니다.
chat_chain = chat_prompt | llm | StrOutputParser()

## call vectorDB

In [5]:
import os
import pandas as pd
import faiss
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Function to create the output directory if not exists
output_dir = 'vectordb/categorywise/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# KosimCSE 모델 및 토크나이저 로드 (모델명 GPU 이동)
model = AutoModel.from_pretrained('BM-K/KosimCSE-bert-multitask').cuda()
tokenizer = AutoTokenizer.from_pretrained('BM-K/kosimCSE-bert-multitask')

def get_embeddings_in_batches(texts, batch_size=8192):
    dataset = TextDataset(texts)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    embeddings_list = []
    model.eval()

    with torch.no_grad():
        for batch_texts in tqdm(data_loader, desc="Processing batches", leave=True):
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to('cuda')

            # Mixed Precision 사용해 배치 연산 수행
            with torch.cuda.amp.autocast():  # FP16에서 실행 후 추출
                outputs = model(**inputs).last_hidden_state[:, 0, :]  # 문장 임베딩 추출
                embeddings = outputs.cpu().numpy()

            embeddings_list.append(embeddings)

    return np.vstack(embeddings_list)

/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## test dataset

In [6]:

unique_ing = []
unique_food_typ = []

for ori in df['original']:
    for oo in eval(ori):
        unique_ing.append(oo)

unique_food_typ = list(set(df['food_typ_nm']))

In [14]:
import os
import faiss
import pandas as pd
from difflib import get_close_matches


from langchain_community.llms import Ollama


def pm1_llm(query, category_dir):
    first_element = query.split(',')[0].strip()

    # Set paths for index and metadata files
    index_file = os.path.join(category_dir, f"{first_element}_faiss_index.bin")
    metadata_file = os.path.join(category_dir, f"{first_element}_metadata.csv")

    # Check if metadata file exists
    if os.path.exists(metadata_file):
        print(f"'{metadata_file}' 파일을 찾았습니다.")
        
        # Check if the FAISS index file exists
        if os.path.exists(index_file):
            # Load FAISS index and metadata
            index = faiss.read_index(index_file)
            meta_df = pd.read_csv(metadata_file)
            meta_df.columns = ['original']
            metadatas = meta_df['original'].tolist()

            # Generate query embedding
            query_embedding = get_embeddings_in_batches([query])

            # Search FAISS index
            D, I = index.search(query_embedding, k=1)

            # Return the most similar metadata entry
            print(f"가장 유사한 original 값: {metadatas[I[0][0]]}")
            
            # Prepare user input for LLM
            user_input = f"""Q: 원본 식재료 {metadatas[I[0][0]]}의 카테고리는 {first_element}야. {query.split(', ')[1:]}은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘."""
            
            # Execute instruction with LLM
            instruction = """너는 식품 개발자야. 형태 형식을 example들과 반드시 같도록 수정해줘."""

            # Q와 A를 모두 제공하는 예시 (형식 강화)
            example_1 = """Q: 원본 식재료 ['토마토', '아보카도', '적양파', '바질']의 카테고리는 소스류야. ['아보카도', '적양파']는 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
            A: 레드 벨 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%"""

            example_2 = """Q: 원본 식재료 ['올리브 오일', '발사믹 식초', '레몬즙']의 카테고리는 소스류야. ['올리브 오일']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
            A: 올리브 오일: 50%, 레드 와인 식초: 30%, 시트러스 주스: 20%"""

            example_3 = """Q: 원본 식재료 ['감자', '당근', '양파', '마늘', '파슬리', '버터']의 카테고리는 스튜야. ['감자', '당근']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
            A: 감자: 35%, 당근: 20%, 샬롯: 15%, 생강: 10%, 고수: 10%, 올리브 오일: 10%"""

            example_4 = """Q: 원본 식재료 ['돼지고기', '간장', '설탕', '생강', '파', '마늘']의 카테고리는 조림류야. ['간장', '생강']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
            A: 닭고기: 40%, 간장: 20%, 꿀: 15%, 생강: 10%, 샬롯: 10%, 미림: 5%"""

            example_5 = """Q: 원본 식재료 ['쌀', '김', '참기름', '계란', '간장', '소금']의 카테고리는 밥류야. ['김', '간장', '소금']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
            A: 현미: 30%, 김: 15%, 들기름: 20%, 메추리알: 10%, 간장: 15%, 소금: 10%"""

            import time

            # 시작 시간 기록
            start_time = time.time()

            try:
                # 모델 호출
                response = llm.invoke(f"{instruction}\n{example_1}\n{example_2}\n{example_3}\n{example_4}\n{example_5}\n{user_input}")

                # 모델의 응답이 너무 오래 걸리는지 확인 (10초 이상 걸리면 경고)
                if time.time() - start_time > 30:
                    print("응답이 예상보다 오래 걸리고 있습니다.")

                print("모델 응답:", response)

            except Exception as e:
                print(f"오류 발생: {e}")
            
            return query.split(','), eval(metadatas[I[0][0]]), list(response)

        else:
            # Handle case where the FAISS index file is missing
            print(f"'{index_file}'에 해당하는 FAISS 인덱스 파일이 없습니다. 패스합니다.")
            return None, None, None
    else:
        # Handle case where the metadata file is missing
        print(f"'{metadata_file}' 파일을 찾을 수 없습니다. 다른 카테고리를 시도해보세요.")
        return None, None, None

In [15]:
from langchain_community.llms import Ollama

def pm1_llm_re_inf(A_in_tmp_res, tmp_inging):

    instruction = f"""
    너는 식품개발자야.
    예제와 같은 형태로 출력해줘.
    """

    example_1 = """Q: 원본 식재료 ['토마토', '아보카도', '적양파', '바질']의 카테고리는 소스류야. 새로운 식재료로 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.
    A: 레드 벨 페퍼: 40%, 아보카도: 30%, 적양파: 20%, 파슬리: 10%"""
    
    user_input = f"""Q: {', '.join(A_in_tmp_res)}에 {', '.join(tmp_inging)}을 추가해서 배합비를 다시 생성하고, 배합비를 100%로 맞춰줘."""

    import time

    # 시작 시간 기록
    start_time = time.time()

    try:
        response = llm.invoke(f"{instruction}\n{example_1}\n{user_input}")

        # 모델의 응답이 너무 오래 걸리는지 확인 (10초 이상 걸리면 경고)
        if time.time() - start_time > 30:
            print("응답이 예상보다 오래 걸리고 있습니다.")

        print("모델 응답:", response)

    except Exception as e:
        print(f"오류 발생: {e}")

    return list(response)

In [16]:
import json
import os
import random
import re

category_dir = 'vectordb/categorywise'  # 카테고리 파일 경로 지정
file_path = 'test_result_1K_2.json'  # JSON 파일 경로

# 파일이 없을 때 빈 리스트로 초기화
if not os.path.exists(file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump([], f, ensure_ascii=False, indent=4)

for i in range(1000):
    tmp_food_typ = random.choice(unique_food_typ)
    tmp_ing = random.sample(unique_ing, 3)

    tmp_str = f"{tmp_food_typ}, {', '.join(tmp_ing)}"

    tmp_input, tmp_sim_input, tmp_res = pm1_llm(tmp_str, category_dir)

    # print(type(tmp_input))
    # print(tmp_sim_input)
    # print(tmp_res[0][1].split('A:')[-1].strip())

    if not tmp_res or len(tmp_res) == 0 or not tmp_res[0] or len(tmp_res[0]) <= 1:
        continue  # 조건이 만족되지 않으면 다음 반복으로 넘어감

    tmp_inging = []
    A_in_tmp_res = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', tmp_res[0][1].split(':A')[-1].strip())
    for inging in tmp_input[1:]:
        if inging.strip() not in A_in_tmp_res:
            tmp_inging.append(inging)

    if len(tmp_inging) != 0:
        tmp_res = pm1_llm_re_inf(A_in_tmp_res, tmp_inging)

    # 새로 추가할 데이터
    new_data = {
        "Input": tmp_input,
        "Similar Input": tmp_sim_input,
        "Response": tmp_res
    }

    # 기존 데이터를 읽어서 새로운 데이터를 append
    with open(file_path, 'r+', encoding='utf-8') as f:
        try:
            # 파일이 비어있는 경우를 처리
            file_content = f.read().strip()
            if file_content:  # 파일에 내용이 있을 때만 로드
                data = json.loads(file_content)  # 기존 데이터 읽기
            else:
                data = []  # 비어있으면 빈 리스트로 초기화
        except json.JSONDecodeError:
            # 파일이 JSON 형식이 아니거나 손상된 경우 처리
            data = []

        data.append(new_data)  # 새로운 데이터를 추가
        f.seek(0)  # 파일 포인터를 다시 파일의 처음으로 이동
        json.dump(data, f, ensure_ascii=False, indent=4)  # 업데이트된 데이터를 다시 저장
        f.truncate()  # 파일의 남은 부분을 잘라냄

'vectordb/categorywise/김치류_metadata.csv' 파일을 찾았습니다.


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

가장 유사한 original 값: ['절임무', '고춧가루', '마늘', '정제수', '생강', '식염', '밀분해추출물', '정백당', '양파', '결정과당', '물엿', '홍고추']
모델 응답: content="Q: 원본 식재료 ['절임무', '고춧가루', '마늘', '정제수', '생강', '식염', '밀분해추출물', '정백당', '양파', '결정과당', '물엿', '홍고추']의 카테고리는 김치류야. ['정제소금', '백설탕', '옥수수전분']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\nA: 절임무: 45%, 고춧가루: 20%, 마늘: 10%, 생강: 5%, 양파: 5%, 옥수수전분: 10%, 정제소금: 5%, 백설탕: 5%" additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-33331038-4221-4829-8ed1-c74219eed48f-0'
모델 응답: content='\r\nQ: 절임무: 45%, 고춧가루: 20%, 마늘: 10%, 생강: 5%, 양파: 5%, 옥수수전분: 10%, 정제소금: 5%, 백설탕: 5%에  정제소금,  백설탕,  옥수수전분을 추가해서 배합비를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\n\r\nA: 절임무: 42.86%, 고춧가루: 17.14%, 마늘: 8.57%, 생강: 3.93%, 양파: 3.93%, 옥수수전분: 10%, 정제소금: 5.71%, 백설탕: 5.71%' additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-4bec9aac-1439-4b41-9750-9289a89e1943-0'
'vectordb/categorywise/유지류_metadata.csv' 파일을 찾았습니다.


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

...

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

가장 유사한 original 값: ['육개장다대기', '육개장볶음다대기', '정제염', '핵산아이지', '비프엑기스분말', '파프리카추출색소', '쇠고기육수베이스', '개량한식간장', '마늘추출농축액p', '양파엑기스j', '무추출물분말', '비프복합베이스1호', '흑후추가루', '대파', '양지엑기스', '쇠고기추출분말p', '감칠맛베이스', '아미노베이스ps', '펩리치고', '막사롬셀렉트파우다효모추출물', '아로마일드', '고추기름1', '토란대', '사골추출농축액', '쇠고기', '정백당', '고춧가루']
모델 응답: content="Q: 원본 식재료 ['육개장다대기', '육개장볶음다대기', '정제염', '핵산아이지', '비프엑기스분말', '파프리카추출색소', '쇠고기육수베이스', '개량한식간장', '마늘추출농축액p', '양파엑기스j', '무추출물분말', '비프복합베이스1호', '흑후추가루', '대파', '양지엑기스', '쇠고기추출분말p', '감칠맛베이스', '아미노베이스ps', '펩리치고', '막사롬셀렉트파우다효모추출물', '아로마일드', '고추기름1', '토란대', '사골추출농축액', '쇠고기', '정백당', '고춧가루']의 카테고리는 즉석조리야. \r\n['치킨추출육수', '유산균배양액혼합분말', '참맛시즈닝jjh']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\nA: 치킨 육수: 35%, 유산균 혼합 분말: 20%, 참맛 시즈닝: 15%, 마늘 추출 농축액: 10%, 양파 추출물: 10%, 무 추출물: 5%, 비프 복합 베이스: 5%, 후추 가루: 3%, 대파: 3%, 양지 추출물: 2%, 감칠맛 베이스: 2%, 아미노 베이스: 2%, 효모 추출물: 1%, 고춧가루: 1%" additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-c0ed3e8b-6824-4172-a74b-319d70ed210e-0'
응답이 예상보다 오

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

가장 유사한 original 값: ['양조간장', '굴소스', '스크램블드에그p', '참기름', '당근', '양파', '양배추', '청피망', '애호박', '홍파프리카', '현미', '보리', '마늘', '정제소금', '대두유', '대두단백', '닭가슴살']
모델 응답: content="Q: 원본 식재료 ['양조간장', '굴소스', '스크램블드에그p', '참기름', '당근', '양파', '양배추', '청피망', '애호박', '홍파프리카', '현미', '보리', '마늘', '정제소금', '대두유', '대두단백', '닭가슴살']의 카테고리는 당뇨환자용식단형식품야. ['참기름', '하얀설탕', '염화마그네슘']은 포함하고 나머지 식재료는 대체하여 식재료 리스트를 다시 생성하고, 배합비를 100%로 맞춰줘.\r\nA: 참기름: 25%, 하얀 설탕: 10%, 염화 마그네슘: 10%, 당근: 15%, 양파: 10%, 양배추: 10%, 청피망: 10%, 애호박: 10%, 홍파프리카: 10%, 현미: 20%, 보리: 10%, 마늘: 5%, 정제 소금: 5%" additional_kwargs={} response_metadata={'finish_reason': 'stop'} id='run-fb6d29ab-76c3-4dd9-a408-adfeed89af41-0'


KeyboardInterrupt: 

In [93]:
# import json
# import os
# import random
# import re

# category_dir = 'vectordb/categorywise'  # 카테고리 파일 경로 지정
# file_path = 'test_result_1K_2.json'  # JSON 파일 경로

# # 파일이 없을 때 빈 리스트로 초기화
# if not os.path.exists(file_path):
#     with open(file_path, 'w', encoding='utf-8') as f:
#         json.dump([], f, ensure_ascii=False, indent=4)

# for i in range(1000):
#     tmp_food_typ = random.choice(unique_food_typ)
#     tmp_ing = random.sample(unique_ing, 3)

#     tmp_str = f"{tmp_food_typ}, {', '.join(tmp_ing)}"

#     tmp_input, tmp_sim_input, tmp_res = pm1_llm(tmp_str, category_dir)

#     tmp_inging = []
#     A_in_tmp_res = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', tmp_res[0][1].split(':A')[-1].strip())
#     # A_in_tmp_res = re.findall(r'[가-힣a-zA-Z0-9]+:\s*\d+%', tmp_res['Response'][0][1].split('A:')[-1].strip())
#     for inging in tmp_input[1:]:
#         if inging.strip() not in A_in_tmp_res:
#             tmp_inging.append(inging)

#     if len(tmp_inging) != 0:
#         tmp_res = pm1_llm_re_inf(A_in_tmp_res, tmp_inging)

#     # 새로 추가할 데이터
#     new_data = {
#         "Input": tmp_input,
#         "Similar Input": tmp_sim_input,
#         "Response": tmp_res
#     }

#     # 기존 데이터를 읽어서 새로운 데이터를 append
#     with open(file_path, 'r+', encoding='utf-8') as f:
#         try:
#             # 파일이 비어있는 경우를 처리
#             file_content = f.read().strip()
#             if file_content:  # 파일에 내용이 있을 때만 로드
#                 data = json.loads(file_content)  # 기존 데이터 읽기
#             else:
#                 data = []  # 비어있으면 빈 리스트로 초기화
#         except json.JSONDecodeError:
#             # 파일이 JSON 형식이 아니거나 손상된 경우 처리
#             data = []

#         data.append(new_data)  # 새로운 데이터를 추가
#         f.seek(0)  # 파일 포인터를 다시 파일의 처음으로 이동
#         json.dump(data, f, ensure_ascii=False, indent=4)  # 업데이트된 데이터를 다시 저장
#         f.truncate()  # 파일의 남은 부분을 잘라냄


In [ ]:
import pandas as pd
import faiss
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

merged_df['combined_str'] = merged_df.apply(lambda x: ', '.join([x['food_typ_nm']] + list(x['mat_nm1'])), axis=1)

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

model = AutoModel.from_pretrained('BM-K/KoSimCSE-bert-multitask').cuda()
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-bert-multitask')

scaler = GradScaler()

def get_embeddings_in_batches(texts, batch_size=8192):  
    dataset = TextDataset(texts)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    embeddings_list = []
    model.eval() 
    
    with torch.no_grad():
        for batch_texts in tqdm(data_loader, desc="Processing batches", leave=True): 
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to('cuda')

            with autocast():
                outputs = model(**inputs).last_hidden_state[:, 0, :] 

            embeddings = outputs.cpu().numpy()
            embeddings_list.append(embeddings)
    
    return np.vstack(embeddings_list)  

texts = merged_df['combined_str'].tolist()
metadatas = merged_df['original'].tolist()

batch_size = 8192  #
embeddings = get_embeddings_in_batches(texts, batch_size=batch_size)

d = embeddings.shape[1] 
index = faiss.IndexFlatL2(d)  

index.add(embeddings)

faiss.write_index(index, "vectordb/faiss_index.bin")

pd.DataFrame(metadatas).to_csv("vectordb/metadata.csv", index=False)

/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jaesung/anaconda3/envs/pm1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_1693930/441785424.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Processing batches:   0%|          | 0/534 [00:00<?, ?it/s]/tmp/ipykernel_1693930/441785424.py:44: FutureWarning: `torch.cuda.amp.

In [ ]:
index = faiss.read_index("vectordb/faiss_index.bin")

meta_df = pd.read_csv("vectordb/metadata.csv")
meta_df = meta_df.rename(columns={'0': 'original'})
metadatas = meta_df['original'].to_list()

query = "떡류,맵쌀,가당백앙금,냉동쑥" 
query_embedding = get_embeddings_in_batches([query])  

D, I = index.search(query_embedding, k=1)

print(f"가장 유사한 original 값: {metadatas[I[0][0]]}")


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZER

가장 유사한 original 값: ['멥쌀', '가당백앙금', '냉동쑥', '한주꽃소금', '쑥가루', '대두유', '정제수']


In [38]:
# from langchain_experimental.agents import create_pandas_dataframe_agent
# from langchain.chat_models import ChatOpenAI
# from langchain.agents.agent_types import AgentType


# # 에이전트 생성
# agent = create_pandas_dataframe_agent(
#     ChatOpenAI(
#         api_key="ollama",
#         model="EEVE-Korean-10.8B:latest",
#         base_url="http://localhost:11434/v1",
#         temperature=0, 
#         streaming=True,
#     ),        # 모델 정의
#     df,                                    # 데이터프레임
#     verbose=True,                          # 추론과정 출력
#     agent_type=AgentType.OPENAI_FUNCTIONS, # AgentType.ZERO_SHOT_REACT_DESCRIPTION
#     allow_dangerous_code=True,
# )

In [ ]:
## llm -> 원래 식품을 창의적으로 잘 대체하게 만들어주면 될 것 같은데? 

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser   #출력을 파싱하여 문자열 형태로 만드는 클래스
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  #langchain_core.prompts로부터 프롬프트를 관리하는 클래스를 가져옮

# llm = ChatOllama(model="EEVE-Korean-10.8B:latest")
llm = ChatOpenAI(
        api_key="ollama",
        model="EEVE-Korean-10.8B:latest",
        base_url="http://localhost:11434/v1",
        temperature=0,
        streaming=True    
    ) 

prompt_prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

chain = prompt_prompt | llm | StrOutputParser()

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [

        (
            "system",
            "You are a helpful AI Assistant. You must answer in Korean.",
        ),

        MessagesPlaceholder(variable_name="messages")
    ]
)

chat_chain = chat_prompt | llm | StrOutputParser()

In [12]:
from langchain_community.llms import Ollama

# llm.invoke("토마토, 아보카도, 적양파, 바질, 올리브 오일, 발사믹 식초, 레몬즙는 샐러드입니다. 각 원재료가 얼마나 포함되어야 하는지 배합비율을 알려주세요.")
llm.invoke(f"{filtered_lists}의 카테고리는 떡류야. 이 식재료 리스트의 세 번째, 네 번째 요소만 대체하여 식재료 리스트를 다시 생성해줘. 다음으로, 이 식재료들의 배합비만으로 구성된 리스트도 생성해줘. 배합비의 합은 100%야.")

AIMessage(content="식재료 리스트의 세 번째와 네 번째 요소를 대체한 새로운 식재료 리스트는 다음과 같습니다:\n\n['멥쌀', '가당백앙금', '냉동쑥', '한주꽃소금', '쑥가루', '대두유', '정제수'] -> ['멥쌀', '가당백앙금', '냉동쑥', '한주꽃소금', '쑥가루', '참기름', '물']\n\n식재료들의 배합비만으로 구성된 리스트는 다음과 같습니다:\n\n['멥쌀', 50%, '가당백앙금', 20%, '냉동쑥', 10%, '한주꽃소금', 5%, '쑥가루', 5%, '참기름', 5%, '물', 5%]", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-01f15862-c833-40a4-9342-9ebc2d350327-0')